In [2]:
import numpy as np
import random
import copy

# ↓問題生成

In [35]:
def puzzle_generation(puzzle_board, issue):
    #ファイル読み込み
    f = open(issue, 'r', encoding='UTF-8')
    data = f.readlines()
    f.close()


    #問題フォーマットの1行目の読み取り ----------------------------------------------------------------------------------------
    s0 = data[0].rstrip('\n').rstrip(',')
    l0 = [int(x.strip()) for x in s0.split(',')]

    colmun = l0[0]              #列
    row = l0[1]                 #行
    selection_cost = l0[2]      #選択コスト


    
    #スライドパズルの盤面の読み取り ------------------------------------------------------------------------------------------
    for i in range(1, row+1):
        row_data = data[i].rstrip('\n').rstrip(',')
        line_data = [int(x.strip()) for x in row_data.split(',')]
        row_array = np.array(line_data)
        if i == 1:
            puzzle_board = row_array
        else:
            puzzle_board = np.vstack((puzzle_board, row_array))

    # colmun = 5
    # row = 6
    #ランダム生成 ---------------------------------------------------------------------------------------------------------
    random_num = random.sample(range(1, row*colmun+1), row*colmun)
    random_list = np.array(random_num)
    puzzle_board = random_list.reshape(row, colmun)


    return puzzle_board, colmun, row, selection_cost

# ↓移動関数

In [75]:
#選択画像を上に移動
def up():

IndentationError: expected an indented block (<ipython-input-75-a93b1bc16aef>, line 2)

In [76]:
#選択画像を下に移動
def down():

IndentationError: expected an indented block (<ipython-input-76-873a4cac58dc>, line 2)

In [77]:
#選択画像を左に移動
def left():

IndentationError: expected an indented block (<ipython-input-77-d63f6356a2c4>, line 2)

In [78]:
#選択画像を右に移動
def right():

IndentationError: expected an indented block (<ipython-input-78-ed3cd70b0384>, line 2)

# ↓回答書き込み

In [79]:
def answer_writing(selection_position, exchange_times, exchange_operation):
    f = open('answer.txt', 'w')

    f.write(str(selection_position[0]) + str(selection_position[1]) + '\n')
    f.write(str(exchange_times) + '\n')
    f.write(exchange_operation + '\n')    

    f.close()

# ↓それぞれの処理

In [36]:
# 2行を除いた盤面の並び替え(たくと)
def puzzle_board_sort_top(puzzle_board, selection_times, exchange_times, exchange_operation):
    # print('colmun(列→)：' + str(colmun))
    # print('row(行↓)：' + str(row))
    # print('selection_cost(選択コスト)：' + str(selection_cost) + '\n')

    # selection_value = colmun * row              #選択値
    # selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
    # selection_x = selection_coord[1]       #選択値のx座標
    # selection_y = selection_coord[0]       #選択値のy座標

    for i in range(row - 2):
        for j in range(colmun):
            target_value = i * colmun + j + 1           #動かしたい値
            selection_value = colmun * row              #選択値

            target_coord = tuple(nd[0] for nd in np.where(puzzle_board == target_value))
            target_x = target_coord[1]                  #動かしたい値のx座標
            target_y = target_coord[0]                  #動かしたい値のy座標
            selection_coord = tuple(nd[0] for nd in np.where(puzzle_board == selection_value))
            selection_x = selection_coord[1]            #選択値のx座標
            selection_y = selection_coord[0]            #選択値のy座標

            print("---------------------------------------------------------------------------------------")
            print("target_value : " + str(target_value))
            # print("selection_value : " + str(selection_value))
            print("target_coord : " + str(target_x + 1) + ", " + str(target_y + 1))
            # print("selection_coord : " + str(selection_x + 1) + ", " + str(selection_y + 1))
            print()



    # print(selection_value)
    # print("x = " + str(selection_x + 1))
    # print("y = " + str(selection_y + 1))


In [81]:
# 下2行の右4マスを除いた盤面の並び替え(たいち)
def puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_times, exchange_operation):
    print("がんば!")

In [82]:
# 右下4ますの並び替え(しのべー)
def puzzle_board_sort_bottom_right(puzzle_board, selection_times, exchange_times, exchange_operation):
    #選択画像変更次には answer_writing を実行して欲しい

    print("がんば!")


In [37]:
# 上3つをまとめたもの
def puzzle_board_sort(puzzle_board):
    selection_times = 0                     #選択回数
    exchange_times = 0                      #交換回数
    exchange_operation = ""                 #交換操作


    #たくと担当
    puzzle_board_sort_top(puzzle_board, selection_times, exchange_times, exchange_operation)
    # #たいち担当
    # puzzle_board_sort_bottom_left(puzzle_board, selection_times, exchange_times, exchange_operation)
    # #しのべー担当
    # puzzle_board_sort_bottom_right(puzzle_board, selection_times, exchange_times, exchange_operation)

# メイン関数

In [38]:
puzzle_board =  np.array([])    #現在の盤面を表す配列

#問題盤面生成
puzzle_board, colmun, row, selection_cost = puzzle_generation(puzzle_board, 'issue/4x4.txt')

print('colmun(列→)：' + str(colmun))
print('row(行↓)：' + str(row))
print('selection_cost(選択コスト)：' + str(selection_cost) + '\n')
print(puzzle_board)


#並び替え
puzzle_board_sort(puzzle_board)

colmun(列→)：5
row(行↓)：6
selection_cost(選択コスト)：2

[[ 1  9 23 20 17]
 [30  6 29 15  3]
 [11 14 26 22  5]
 [28 19  8 10 24]
 [ 2 16  7 21 12]
 [25 18  4 27 13]]
---------------------------------------------------------------------------------------
target_value : 1
target_coord : 1, 1

---------------------------------------------------------------------------------------
target_value : 2
target_coord : 1, 5

---------------------------------------------------------------------------------------
target_value : 3
target_coord : 5, 2

---------------------------------------------------------------------------------------
target_value : 4
target_coord : 3, 6

---------------------------------------------------------------------------------------
target_value : 5
target_coord : 5, 3

---------------------------------------------------------------------------------------
target_value : 6
target_coord : 2, 2

---------------------------------------------------------------------------------------